<a href="https://colab.research.google.com/github/LucaLazar07/VeridionChallenge/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer

**Reading the Data**

In [ ]:
insurance_df = pd.read_csv('ml_insurance_challenge.csv')
label_df = pd.read_csv('insurance_taxonomy.csv')

**Info about the Data**

In [ ]:
insurance_df

In [ ]:
label_df

In [ ]:
insurance_df.describe()

In [ ]:
insurance_df.isnull().sum()

**Text Preprocessing prior to Company Embeddings**

In [ ]:
def create_text_about_company(row):
    if (pd.notna(row['business_tags'])):
        business_tags_processed = row['business_tags'].replace("['", "").replace("']", "").replace("'", "")

    description = row['description'] if pd.notna(row['description']) else ""

    business_tags = business_tags_processed if pd.notna(business_tags_processed) else ""

    sector = row['sector'] if pd.notna(row['sector']) else ""

    category = row['category'] if pd.notna(row['category']) else ""

    niche = row['niche'] if pd.notna(row['niche']) else ""

    text = f"{description} Services: {business_tags}. Industry: {sector} - {category} - {niche}"

    return text


In [ ]:
# testing to see what model to use based on the number of tokens accepted by it

rows = insurance_df.shape[0]
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

for i in range(rows):
    string_row = create_text_about_company(insurance_df.iloc[i])
    tokens = tokenizer.encode(str(string_row))
    if (len(tokens)) > 512:
        print("A model with a bigger number of tokens is needed")
        break

**Model Creation**

In [ ]:
model = SentenceTransformer("BAAI/bge-large-en-v1.5")

company_text = []

for i in range(rows):
    string_row = create_text_about_company(insurance_df.iloc[i])
    company_text.append(string_row)

company_embeddings = model.encode(company_text)
label_embeddings = model.encode(label_df['label'].tolist())

similarity = util.cos_sim(company_embeddings, label_embeddings)

**Veryfing most similar labels for each company**

**Bi-Encoder**

In [ ]:
k = 5
for i in range(rows):
    company_similarity = similarity[i]

    top_k_labels = company_similarity.topk(k=k)

    label_index = top_k_labels.indices[0].item()
    label_name = label_df.iloc[label_index]['label']

    insurance_df.loc[i, "label_insurance"] = label_name

insurance_df.to_csv('ml_insurance_challenge_with_bi_encoder.csv')


**Cross-Encoder**

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

for i in range(rows):
    query = company_text[i]
    corpus = label_df['label'].tolist()

    ranks = model.rank(query, corpus)
    label_name = corpus[ranks[0]['corpus_id']]

    insurance_df.loc[i, "label_insurance"] = label_name

insurance_df.to_csv("ml_insurance_challenge_with_cross_encoder.csv")


**Bi-Encoder + Cross-Encoder**

In [ ]:
k = 10
best_k = 5
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")


for i in range(rows):
    company_similarity = similarity[i]
    query = company_text[i]

    top_k_labels = company_similarity.topk(k=k)
    best_labels = []

    for index in range(best_k):
        label_index = top_k_labels.indices[index].item()
        best_labels.append(label_df.iloc[label_index]['label'])

    corpus = best_labels

    ranks = model.rank(query, corpus)
    label_name = corpus[ranks[0]['corpus_id']]

    insurance_df.loc[i, "label_insurance"] = label_name

insurance_df.to_csv("ml_insurance_challenge_with_bi_and_cross_encoder.csv")


**Results overview**

After analyzing the results from using a bi-encoder, cross-encoder and bi + cross encoder model, I noticed that the best results were obtained by using the bi and the bi + cross encoder. Although, I was not satisfied with the results so I decided to attack the challenge with a new approach. I will change the weight for all the features except the description and I also changed the pretrained models used for embedding and cross-encoder to a DeBERTa-v3 based model who has a much higher performance in these types of situations than the ones used before.

**Weight change for features**

In [ ]:
def create_text_about_company(row):
    business_tags_processed = ""
    if (pd.notna(row['business_tags'])):
        business_tags_processed = row['business_tags'].replace("['", "").replace("']", "").replace("'", "")

    description = row['description'] if pd.notna(row['description']) else ""

    business_tags = business_tags_processed if pd.notna(business_tags_processed) else ""

    sector = row['sector'] if pd.notna(row['sector']) else ""

    category = row['category'] if pd.notna(row['category']) else ""

    niche = row['niche'] if pd.notna(row['niche']) else ""

    text = f"{description} Services: {business_tags}. Industry: {sector} - {category} - {niche}"

    return text

**Using DeBERTa-V3 model for embedding**

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model = AutoModel.from_pretrained("microsoft/deberta-v3-base")

def deberta_embeddings(text, tokenizer, model):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
  with torch.no_grad():
    output = model(**inputs)

  hidden_states = output.last_hidden_state

  mask = inputs['attention_mask'].unsqueeze(-1).float()

  sum_embeddings = torch.sum(hidden_states * mask, dim=1)

  sum_mask = torch.max(torch.sum(mask, dim=1), torch.tensor(1e-9))

  mean_embeddings = sum_embeddings / sum_mask

  return mean_embeddings

**Applying DeBERTa Embedding to the companies and labels**

In [ ]:
rows = insurance_df.shape[0]
company_text = []

for i in range(rows):
    string_row = create_text_about_company(insurance_df.iloc[i])
    company_text.append(string_row)

company_embeddings_list = [deberta_embeddings(text, tokenizer, model) for text in company_text]
company_embeddings = torch.stack(company_embeddings_list).squeeze()

label_embeddings_list = [deberta_embeddings(label, tokenizer, model) for label in label_df['label'].tolist()]
label_embeddings = torch.stack(label_embeddings_list).squeeze()

similarity = util.cos_sim(company_embeddings, label_embeddings)

print(company_embeddings.shape)
print(label_embeddings.shape)
print(similarity.shape)

**Assigning the best label to the company**

In [ ]:
for i in range(rows):
  company_similarity = similarity[i]

  top_labels = company_similarity.topk(k=5)

  label_index = top_labels.indices[0].item()
  label_name = label_df.iloc[label_index]['label']

  insurance_df.loc[i, "label_insurance"] = label_name

insurance_df.to_csv("ml_insurance_challenge_with_deberta.csv")

In [ ]:
deberta_csv = pd.read_csv("ml_insurance_challenge_with_deberta.csv")
deberta_csv['label_insurance'].value_counts()

**Despite its good recognition as a pretrained model, DeBERTav3 failed to assign the labels correctly, assigning the label "Non-Alcoholic Beverage Manufacturing" to 8815 companies. This might be due to the embeddings as they are manually calculated. Therefore, I decided to stick to the results obtained before who have less mistakes.**

In [ ]:
bi_csv = pd.read_csv("ml_insurance_challenge_with_bi_encoder.csv")
bi_plus_cross_csv = pd.read_csv("ml_insurance_challenge_with_bi_and_cross_encoder.csv")

In [ ]:
bi_csv['label_insurance'].value_counts()

In [ ]:
bi_plus_cross_csv['label_insurance'].value_counts()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(bi_csv['label_insurance'], bi_plus_cross_csv['label_insurance']))

**Observing the results from the Bi encoding and the Bi and Cross encoding, we notice that the Bi and Cross encoding tends to assign the label "Non-Alcoholic Beverage Manufacturing" more than the Bi encoding, therefore being less precise. Therefore, the best result is obtained with a Bi encoder.**